In [ ]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os
import sys
import shutil
import time
from PyPDF2 import PdfFileMerger

script_exec_start = time.time()
sns.set()

bat_player_1 = 'V Kohli'
bat_player_2 = 'RG Sharma'
bat_player_3 = 'SPD Smith'

def clean_opponent_names (opponent):
    if opponent == 'Sunrisers Hyderabad' or opponent == 'Deccan Chargers':
        opponent = 'Hyderabad (Sunrisers/DC)'
    elif opponent == 'Rising Pune Supergiant' or opponent == 'Rising Pune Supergiants' or opponent == 'Pune Warriors':
        opponent = 'Pune (RPS/Warriors)'
    elif opponent == 'Delhi Daredevils' or opponent == 'Delhi Capitals':
        opponent = 'Delhi (Capitals/DD)'
    elif opponent == 'Chennai Super Kings':
        opponent = 'Chennai (CSK)' 
    elif opponent == 'Royal Challengers Bangalore':
        opponent = 'Bangalore (RCB)' 
    elif opponent == 'Kings XI Punjab':
        opponent = 'Punjab (KXI)' 
    return opponent

df_matches = pd.read_csv('matches.csv')
df_matches['year'] = pd.DatetimeIndex(df_matches['date']).year
df_matches['month'] = pd.DatetimeIndex(df_matches['date']).month
df_matches['day'] = pd.DatetimeIndex(df_matches['date']).day
df_matches['day_of_week'] = pd.DatetimeIndex(df_matches['date']).day_name()
df_deliveries = pd.read_csv('deliveries.csv')
#df_deliveries = df_deliveries[df_deliveries['is_super_over'] == 0]
df_deliveries = df_deliveries[(df_deliveries['is_super_over'] == 0) & (df_deliveries['inning'] <= 2)]


df_deliveries['batting_team'] = df_deliveries['batting_team'].apply(lambda x: clean_opponent_names(x))
df_deliveries['bowling_team'] = df_deliveries['bowling_team'].apply(lambda x: clean_opponent_names(x))
df_matches['team1'] = df_matches['team1'].apply(lambda x: clean_opponent_names(x))
df_matches['team2'] = df_matches['team2'].apply(lambda x: clean_opponent_names(x))
df_matches['toss_winner'] = df_matches['toss_winner'].apply(lambda x: clean_opponent_names(x))
df_matches['winner'] = df_matches['winner'].apply(lambda x: clean_opponent_names(x))



def create_sub_folder (folder_name):
    if os.path.exists(cwd + '/' + folder_name):
        shutil.rmtree(cwd + '/' + folder_name)
    os.mkdir(cwd + '/' + folder_name)

def sort_unique(seq):
    sort = set()
    sort_add = sort.add
    return [x for x in seq if not (x in sort or sort_add(x))]

def find_bat_pos(match_id, inning, player_name):
    df_del = df_deliveries[(df_deliveries['match_id'] == match_id) & (df_deliveries['inning'] == inning)]
    a = df_del['batsman'].tolist()
    b = df_del['non_striker'].tolist()
    ordered_bat_pos = []
    for i in range(0,max(len(a), len(b))):
        try:
            ordered_bat_pos.append(a[i])
        except:
            print('')
        try:
            ordered_bat_pos.append(b[i])
        except:
            continue
    ordered_bat_pos = sort_unique(ordered_bat_pos)
    pos = ordered_bat_pos.index(player_name)+1
    return pos


def find_bat_team(match_id, inning, player_name):
    df_del = df_deliveries[(df_deliveries['match_id'] == match_id) & (df_deliveries['inning'] == inning) & \
                          (df_deliveries['batsman'] == player_name)]
    return df_del['batting_team'].tolist()[0]

def find_bat_opp_team(match_id, inning, player_name):
    df_del = df_deliveries[(df_deliveries['match_id'] == match_id) & (df_deliveries['inning'] == inning) & \
                          (df_deliveries['batsman'] == player_name)]
    return df_del['bowling_team'].tolist()[0]

def find_team_result(match_id, team, player_name):
    df_winner = df_matches[df_matches['id'] == match_id]
    winner = df_winner['winner'].tolist()[0]
    if winner == team:
        result = 'Won'
    else:
        result = 'Lost'
    return result

def find_how_out(match_id, inning, player_name):
    df_del = df_deliveries[(df_deliveries['match_id'] == match_id) & (df_deliveries['inning'] == inning) & \
                (df_deliveries['batsman'] == player_name) & (df_deliveries['player_dismissed'] == player_name)]
    if len(df_del) > 0:
        how_out = df_del['dismissal_kind'].tolist()[0]
    else:
        how_out = 'Not Out'
    return how_out


def load_bat_player(bat_player):
    df_bat_player = pd.DataFrame()

    df_bat_player = df_deliveries[df_deliveries['batsman']== bat_player].groupby('match_id').sum()

    
    df_bat_player['batsman_balls'] = df_deliveries[df_deliveries['batsman']== bat_player][['match_id', 'ball']].\
                                     groupby('match_id').count()
    df_bat_player['player_dismissed'] = df_deliveries[df_deliveries['batsman']== bat_player][['match_id', 'player_dismissed']].\
                                     groupby('match_id').count()
    df_bat_player['inning'] = df_deliveries[df_deliveries['batsman']== bat_player][['match_id', 'inning']].\
                                     groupby('match_id').mean()

    df_bat_player['strike_rate'] = round(df_bat_player['batsman_runs']*100/df_bat_player['batsman_balls'], 2)
    
    
    df_deliveries[df_deliveries['batsman']== bat_player].groupby('match_id').count()
    df_bat_player['bat_position'] = df_bat_player.index

    
    df_bat_player['team'] = df_bat_player.index
    df_bat_player['team'] = df_bat_player.apply\
    (lambda x: find_bat_team(x['team'], x['inning'], bat_player), axis=1)
 
    df_bat_player['opponent'] = df_bat_player.index
    df_bat_player['opponent'] = df_bat_player.apply\
    (lambda x: find_bat_opp_team(x['opponent'], x['inning'], bat_player), axis=1)
 
    df_bat_player['result'] = df_bat_player.index
    df_bat_player['result'] = df_bat_player.apply(lambda x: find_team_result(x['result'], x['team'], bat_player), \
                                                  axis=1)

    df_bat_player['dismissal_kind'] = df_bat_player.index
    df_bat_player['dismissal_kind'] = df_bat_player.apply\
        (lambda x: find_how_out(x['dismissal_kind'], x['inning'], bat_player), axis=1)

    df_bat_player['bat_position'] = df_bat_player.apply\
    (lambda x: find_bat_pos(x['bat_position'], x['inning'], bat_player), axis=1)

    df_bat_player = pd.merge(df_bat_player, df_matches[['id','city','date', 'year', 'month', 'day', 'day_of_week']] ,\
                               left_on='match_id', right_on = 'id', how='left')
    df_bat_player = df_bat_player.rename(columns = {'id': 'match_id'})
    
    return df_bat_player

lastnames = []
df_batplayer_1 = load_bat_player(bat_player_1)
df_batplayer_2 = load_bat_player(bat_player_2)
df_batplayer_3 = load_bat_player(bat_player_3)

lastnames.append(bat_player_1.split(' ')[-1])
lastnames.append(bat_player_2.split(' ')[-1])
lastnames.append(bat_player_3.split(' ')[-1])

pdfs = []
xm = 80
ym = 150

def plot_runs_balls_pos(df_player):
    df_player = df_player[df_player['bat_position'].apply(lambda x: str(x).isnumeric())]
    x = df_player['batsman_balls']
    y = df_player['batsman_runs']
    mat = len(df_player)
    win = len(df_player[df_player['result'] == 'Won'])
    
    one=len(df_player[df_player['bat_position'] == 1])
    two=len(df_player[df_player['bat_position'] == 2])+one
    three=len(df_player[df_player['bat_position'] == 3])+two
    four=len(df_player[df_player['bat_position'] == 4])+three
    five=len(df_player[df_player['bat_position'] == 5])+four
    six=len(df_player[df_player['bat_position'] >= 6])+five
    return x, y, mat, win, one, two, three, four, five, six

def plot_avg_player(df_player):
    df_player_win = df_player[df_player['result'] == 'Won']
    df_player_lost = df_player[df_player['result'] != 'Won']
    df_player_first = df_player[df_player['inning'] == 1]
    df_player_second = df_player[df_player['inning'] != 1]
    df_player_weekend = df_player[(df_player['day_of_week'] == 'Friday') | (df_player['day_of_week'] == 'Saturday')\
                                 | (df_player['day_of_week'] == 'Sunday')]
    df_player_weekday = df_player[(df_player['day_of_week'] != 'Friday') | (df_player['day_of_week'] != 'Saturday')\
                                 | (df_player['day_of_week'] != 'Sunday')]
    b1 = []
    b2 = []
    b3 = []
    b1.append(df_player['batsman_runs'].sum()/(len(df_player)-len(df_player[df_player['player_dismissed'] == 0])))
    b1.append(b1[0])
    b1.append(b1[0])  
    count = len(df_player)
    mean = b1[0]
    std = df_player['batsman_runs'].std()
    yerr = 1.96*std/np.sqrt(count)
    
    b2.append(df_player_win['batsman_runs'].sum()/\
              (len(df_player_win)-len(df_player_win[df_player_win['player_dismissed'] == 0])))
    b2.append(df_player_first['batsman_runs'].sum()/\
              (len(df_player_first)-len(df_player_first[df_player_first['player_dismissed'] == 0])))
    b2.append(df_player_weekend['batsman_runs'].sum()/\
              (len(df_player_weekend)-len(df_player_weekend[df_player_weekend['player_dismissed'] == 0])))
    
    b3.append(df_player_lost['batsman_runs'].sum()/\
              (len(df_player_lost)-len(df_player_lost[df_player_lost['player_dismissed'] == 0])))
    b3.append(df_player_second['batsman_runs'].sum()/\
              (len(df_player_second)-len(df_player_second[df_player_second['player_dismissed'] == 0])))
    b3.append(df_player_weekday['batsman_runs'].sum()/\
              (len(df_player_weekday)-len(df_player_weekday[df_player_weekday['player_dismissed'] == 0])))
    return b1,b2,b3,yerr

def plot_sr_player(df_player):
    df_player_win = df_player[df_player['result'] == 'Won']
    df_player_lost = df_player[df_player['result'] != 'Won']
    df_player_first = df_player[df_player['inning'] == 1]
    df_player_second = df_player[df_player['inning'] != 1]
    df_player_weekend = df_player[(df_player['day_of_week'] == 'Friday') | (df_player['day_of_week'] == 'Saturday')\
                                 | (df_player['day_of_week'] == 'Sunday')]
    df_player_weekday = df_player[(df_player['day_of_week'] != 'Friday') | (df_player['day_of_week'] != 'Saturday')\
                                 | (df_player['day_of_week'] != 'Sunday')]
    b1 = []
    b2 = []
    b3 = []
    b1.append((df_player['batsman_runs'].sum()/df_player['batsman_balls'].sum()) * 100)
    b1.append(b1[0])
    b1.append(b1[0])  
    count = len(df_player)
    mean = b1[0]
    std = df_player['batsman_runs'].std()
    yerr = 1.96*std/np.sqrt(count)
    b2.append((df_player_win['batsman_runs'].sum()/df_player_win['batsman_balls'].sum()) * 100)
    b2.append((df_player_first['batsman_runs'].sum()/df_player_first['batsman_balls'].sum()) * 100)
    b2.append((df_player_weekend['batsman_runs'].sum()/df_player_weekend['batsman_balls'].sum()) * 100)
    
    b3.append((df_player_lost['batsman_runs'].sum()/df_player_lost['batsman_balls'].sum()) * 100)
    b3.append((df_player_second['batsman_runs'].sum()/df_player_second['batsman_balls'].sum()) * 100)
    b3.append((df_player_weekday['batsman_runs'].sum()/df_player_weekday['batsman_balls'].sum()) * 100)
    return b1,b2,b3,yerr


plt.figure(figsize = (20,15), edgecolor = 'b')
ax1 = plt.subplot(3, 3, 1)
x, y, mat, win, one, two, three, four, five, six = plot_runs_balls_pos(df_batplayer_1)
ax1 = plt.xlabel('Balls Faced', fontsize=12)
ax1 = plt.ylabel('Runs Scored', fontsize=15)
ax1 = plt.scatter(x[:two], y[:two], color='red')
ax1 = plt.scatter(x[two:three], y[two:three], color='green')
ax1 = plt.scatter(x[three:four], y[three:four], color='blue')
ax1 = plt.scatter(x[four:five], y[four:five], color='orange')
ax1 = plt.scatter(x[five:], y[five:], color='black')
ax1 = plt.legend(['Opening','No: 3', 'No: 4', 'No: 5', 'No:6 +'], loc='upper left')
ax1 = plt.title(bat_player_1 + '  (M:' + str(mat) + ', W:' + str(win) + ', w%:' + str(round(win*100/mat,1)) + ')', \
                                                                                size=17, color='blue')
ax1 = plt.axis(xmin=0, xmax=xm)
ax1 = plt.axis(ymin=0, ymax=ym)

ax2 = plt.subplot(3, 3, 2)
x, y, mat, win, one, two, three, four, five, six = plot_runs_balls_pos(df_batplayer_2)
ax2 = plt.scatter(x[:two], y[:two], color='red')
ax2 = plt.scatter(x[two:three], y[two:three], color='green')
ax2 = plt.scatter(x[three:four], y[three:four], color='blue')
ax2 = plt.scatter(x[four:five], y[four:five], color='orange')
ax2 = plt.scatter(x[five:], y[five:], color='black')
ax2 = plt.legend(['Opening','No: 3', 'No: 4', 'No: 5', 'No:6 +'], loc='upper left')
ax2 = plt.title(bat_player_2 + '  (M:' + str(mat) + ', W:' + str(win) + ', w%:' + str(round(win*100/mat,1)) + ')', \
                                                                                size=17, color='blue')
ax2 = plt.axis(xmin=0, xmax=xm)
ax2 = plt.axis(ymin=0, ymax=ym)

ax3 = plt.subplot(3, 3, 3)
x, y, mat, win, one, two, three, four, five, six = plot_runs_balls_pos(df_batplayer_3)
ax3 = plt.scatter(x[:two], y[:two], color='red')
ax3 = plt.scatter(x[two:three], y[two:three], color='green')
ax3 = plt.scatter(x[three:four], y[three:four], color='blue')
ax3 = plt.scatter(x[four:five], y[four:five], color='orange')
ax3 = plt.scatter(x[five:], y[five:], color='black')
ax3 = plt.legend(['Opening','No: 3', 'No: 4', 'No: 5', 'No:6 +'], loc='upper left')
ax3 = plt.title(bat_player_3 + '  (M:' + str(mat) + ', W:' + str(win) + ', w%:' + str(round(win*100/mat,1)) + ')', \
                                                                                size=17, color='blue')
ax3 = plt.axis(xmin=0, xmax=xm)
ax3 = plt.axis(ymin=0, ymax=ym)

ax4 = plt.subplot(3, 3, 4)
x = [1,2,3]
x = np.asarray(x)
bar1, bar2, bar3, yerr = plot_avg_player(df_batplayer_1)
ax4.bar(x-0.2, bar1, width=0.2, yerr = yerr, capsize = 7, color='darkgrey', align='center')
ax4.bar(x, bar2, width=0.2, color='royalblue', align='center')
ax4.bar(x+0.2, bar3, width=0.2, color='palevioletred', align='center')
labels = ['','All-Win-Loss','','All-Set-Chs','','All-WEn-WDy']
ax4.set_ylabel('Average - 95% CI', fontsize=15)
ax4.set_xticklabels(labels)

ax5 = plt.subplot(3, 3, 5, sharey = ax4)
x = [1,2,3]
x = np.asarray(x)
bar1, bar2, bar3, yerr = plot_avg_player(df_batplayer_2)
ax5.bar(x-0.2, bar1, width=0.2, yerr = yerr, capsize = 7, color='darkgrey', align='center')
ax5.bar(x, bar2, width=0.2, color='royalblue', align='center')
ax5.bar(x+0.2, bar3, width=0.2, color='palevioletred', align='center')
labels = ['','All-Win-Loss','','All-Set-Chs','','All-WEn-WDy']
ax5.set_xticklabels(labels)

ax6 = plt.subplot(3, 3, 6, sharey = ax4)
x = [1,2,3]
x = np.asarray(x)
bar1, bar2, bar3, yerr = plot_avg_player(df_batplayer_3)
ax6.bar(x-0.2, bar1, width=0.2, yerr= yerr, capsize = 7, color='darkgrey', align='center')
ax6.bar(x, bar2, width=0.2, color='royalblue', align='center')
ax6.bar(x+0.2, bar3, width=0.2, color='palevioletred', align='center')
labels = ['','All-Win-Loss','','All-Set-Chs','','All-WEn-WDy']
ax6.set_xticklabels(labels)

ax7 = plt.subplot(3, 3, 7)
x = [1,2,3]
x = np.asarray(x)
bar1, bar2, bar3, yerr = plot_sr_player(df_batplayer_1)
ax7.bar(x-0.2, bar1, width=0.2, yerr= yerr, capsize = 7, color='darkgrey', align='center')
ax7.bar(x, bar2, width=0.2, color='lightgreen', align='center')
ax7.bar(x+0.2, bar3, width=0.2, color='sandybrown', align='center')
labels = ['','All-Win-Loss','','All-Set-Chs','','All-WEn-WDy']
ax7.set_ylabel('Strike Rate - 95% CI', fontsize=15)
ax7.set_xticklabels(labels)

ax8 = plt.subplot(3, 3, 8, sharey = ax7)
x = [1,2,3]
x = np.asarray(x)
bar1, bar2, bar3, yerr = plot_sr_player(df_batplayer_2)
ax8.bar(x-0.2, bar1, width=0.2, yerr= yerr, capsize = 7, color='darkgrey', align='center')
ax8.bar(x, bar2, width=0.2, color='lightgreen', align='center')
ax8.bar(x+0.2, bar3, width=0.2, color='sandybrown', align='center')
labels = ['','All-Win-Loss','','All-Set-Chs','','All-WEn-WDy']
ax8.set_xticklabels(labels)

ax9 = plt.subplot(3, 3, 9, sharey = ax7)
x = [1,2,3]
x = np.asarray(x)
bar1, bar2, bar3, yerr = plot_sr_player(df_batplayer_3)
ax9.bar(x-0.2, bar1, width=0.2, yerr= yerr, capsize = 7, color='darkgrey', align='center')
ax9.bar(x, bar2, width=0.2, color='lightgreen', align='center')
ax9.bar(x+0.2, bar3, width=0.2, color='sandybrown', align='center')
labels = ['','All-Win-Loss','','All-Set-Chs','','All-WEn-WDy']
ax9.set_xticklabels(labels)

plt.suptitle('IPL: 3 Batsmen - Analysis Plot', size=25, color='black')

df_batplayer_1['player_name'] = bat_player_1
df_batplayer_2['player_name'] = bat_player_2
df_batplayer_3['player_name'] = bat_player_3

df_batplayer = df_batplayer_1
df_batplayer = df_batplayer.append(df_batplayer_2)
df_batplayer = df_batplayer.append(df_batplayer_3)

cwd = os.getcwd()
create_sub_folder ('Analysis_Results')

plt.savefig(cwd+'/Analysis_Results/01_'+'_'.join(lastnames)+'_Bat_Analysis.png', dpi=400)
plt.savefig(cwd+'/Analysis_Results/01_'+'_'.join(lastnames)+'_Bat_Analysis.pdf', dpi=400)
plt.show();



In [ ]:
g = sns.pairplot(df_batplayer[['batsman_runs', 'batsman_balls', 'strike_rate', 'player_name']], \
                 hue="player_name", height=5)
g.fig.subplots_adjust(top=0.95)
g.fig.suptitle('Runs vs Balls vs Strike Rate - Pairplot', fontsize=25, color='black')
g.fig.savefig(cwd+'/Analysis_Results/02_'+'_'.join(lastnames)+'_Bat_Analysis.png', dpi=400)
g.fig.savefig(cwd+'/Analysis_Results/02_'+'_'.join(lastnames)+'_Bat_Analysis.pdf', dpi=400)


In [ ]:
plt.figure(figsize = (18,23), edgecolor = 'b')

ax1 = plt.subplot(3, 2, 1)
ax1 = sns.boxplot(df_batplayer_1['batsman_balls'], df_batplayer_1['dismissal_kind']);
ax1 = sns.violinplot(df_batplayer_1['batsman_balls'], df_batplayer_1['dismissal_kind']);


ax2 = plt.subplot(3, 2, 2)
ax2 = sns.violinplot(x='bat_position', y='batsman_runs', data=df_batplayer_1);

ax3 = plt.subplot(3, 2, 3)
ax3 = sns.boxplot(x='batsman_runs', y='city', data=df_batplayer_1);

ax4 = plt.subplot(3, 2, 4)
ax4 = sns.boxplot(x='day_of_week', y='batsman_runs', hue='inning', data=df_batplayer_1, \
                 order = ['Friday', 'Saturday', 'Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday']);

ax5 = plt.subplot(3, 2, 5)
df_bat_1 = df_batplayer_1.groupby(df_batplayer_1['opponent']).sum()
df_bat_1['total_innings'] = df_batplayer_1[['opponent','player_dismissed']].groupby('opponent').count()
df_bat_1['average'] = round(df_bat_1['batsman_runs']/df_bat_1['player_dismissed'],2)
df_bat_1['strike_rate'] = round(df_bat_1['batsman_runs']*100/df_bat_1['batsman_balls'],2)

ax5 = sns.barplot(x=df_bat_1.index, y='batsman_runs', data=df_bat_1, ci=95);
ax5 = sns.lineplot(x=df_bat_1.index, y='average', data=df_bat_1, color='blue', label='Average')
ax5 = sns.lineplot(x=df_bat_1.index, y='strike_rate', data=df_bat_1, color='red', label='Strike Rate')
ax5.set_xticklabels(ax5.get_xticklabels(), rotation=90)
ax5.set_ylabel('total_runs')
ax5.legend()

ax6 = plt.subplot(3, 2, 6)
ax6 = sns.boxplot(x='year', y='batsman_runs', data=df_batplayer_1);
plt.subplots_adjust(top=0.95)
plt.suptitle(bat_player_1 + ': IPL Career Performance', fontsize=28, color='black');

plt.savefig(cwd+'/Analysis_Results/03_'+lastnames[0]+'_Bat_Analysis.png', dpi=400)
plt.savefig(cwd+'/Analysis_Results/03_'+lastnames[0]+'_Bat_Analysis.pdf', dpi=400)

In [ ]:
plt.figure(figsize = (18,23), edgecolor = 'b')

ax1 = plt.subplot(3, 2, 1)
ax1 = sns.boxplot(df_batplayer_2['batsman_balls'], df_batplayer_2['dismissal_kind']);
ax1 = sns.violinplot(df_batplayer_2['batsman_balls'], df_batplayer_2['dismissal_kind']);


ax2 = plt.subplot(3, 2, 2)
ax2 = sns.violinplot(x='bat_position', y='batsman_runs', data=df_batplayer_2);

ax3 = plt.subplot(3, 2, 3)
ax3 = sns.boxplot(x='batsman_runs', y='city', data=df_batplayer_2);

ax4 = plt.subplot(3, 2, 4)
ax4 = sns.boxplot(x='day_of_week', y='batsman_runs', hue='inning', data=df_batplayer_2, \
                 order = ['Friday', 'Saturday', 'Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday']);

ax5 = plt.subplot(3, 2, 5)
df_bat_2 = df_batplayer_2.groupby(df_batplayer_2['opponent']).sum()
df_bat_2['total_innings'] = df_batplayer_2[['opponent','player_dismissed']].groupby('opponent').count()
df_bat_2['average'] = round(df_bat_2['batsman_runs']/df_bat_2['player_dismissed'],2)
df_bat_2['strike_rate'] = round(df_bat_2['batsman_runs']*100/df_bat_2['batsman_balls'],2)

ax5 = sns.barplot(x=df_bat_2.index, y='batsman_runs', data=df_bat_2, ci=95);
ax5 = sns.lineplot(x=df_bat_2.index, y='average', data=df_bat_2, color='blue', label='Average')
ax5 = sns.lineplot(x=df_bat_2.index, y='strike_rate', data=df_bat_2, color='red', label='Strike Rate')
ax5.set_xticklabels(ax5.get_xticklabels(), rotation=90)
ax5.set_ylabel('total_runs')
ax5.legend()

ax6 = plt.subplot(3, 2, 6)
ax6 = sns.boxplot(x='year', y='batsman_runs', data=df_batplayer_2);
plt.subplots_adjust(top=0.95)
plt.suptitle(bat_player_2 + ': IPL Career Performance', fontsize=28, color='black');

plt.savefig(cwd+'/Analysis_Results/04_'+lastnames[1]+'_Bat_Analysis.png', dpi=400)
plt.savefig(cwd+'/Analysis_Results/04_'+lastnames[1]+'_Bat_Analysis.pdf', dpi=400)

In [ ]:
plt.figure(figsize = (18,23), edgecolor = 'b')

ax1 = plt.subplot(3, 2, 1)
ax1 = sns.boxplot(df_batplayer_3['batsman_balls'], df_batplayer_3['dismissal_kind']);
ax1 = sns.violinplot(df_batplayer_3['batsman_balls'], df_batplayer_3['dismissal_kind']);


ax2 = plt.subplot(3, 2, 2)
ax2 = sns.violinplot(x='bat_position', y='batsman_runs', data=df_batplayer_3);

ax3 = plt.subplot(3, 2, 3)
ax3 = sns.boxplot(x='batsman_runs', y='city', data=df_batplayer_3);

ax4 = plt.subplot(3, 2, 4)
ax4 = sns.boxplot(x='day_of_week', y='batsman_runs', hue='inning', data=df_batplayer_3, \
                 order = ['Friday', 'Saturday', 'Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday']);

ax5 = plt.subplot(3, 2, 5)
df_bat_3 = df_batplayer_3.groupby(df_batplayer_3['opponent']).sum()
df_bat_3['total_innings'] = df_batplayer_3[['opponent','player_dismissed']].groupby('opponent').count()
df_bat_3['average'] = round(df_bat_3['batsman_runs']/df_bat_3['player_dismissed'],2)
df_bat_3['strike_rate'] = round(df_bat_3['batsman_runs']*100/df_bat_3['batsman_balls'],2)

ax5 = sns.barplot(x=df_bat_3.index, y='batsman_runs', data=df_bat_3, ci=95);
ax5 = sns.lineplot(x=df_bat_3.index, y='average', data=df_bat_3, color='blue', label='Average')
ax5 = sns.lineplot(x=df_bat_3.index, y='strike_rate', data=df_bat_3, color='red', label='Strike Rate')
ax5.set_xticklabels(ax5.get_xticklabels(), rotation=90)
ax5.set_ylabel('total_runs')
ax5.legend()

ax6 = plt.subplot(3, 2, 6)
ax6 = sns.boxplot(x='year', y='batsman_runs', data=df_batplayer_3);
plt.subplots_adjust(top=0.95)
plt.suptitle(bat_player_3 + ': IPL Career Performance', fontsize=28, color='black');

plt.savefig(cwd+'/Analysis_Results/05_'+lastnames[2]+'_Bat_Analysis.png', dpi=400)
plt.savefig(cwd+'/Analysis_Results/05_'+lastnames[2]+'_Bat_Analysis.pdf', dpi=400)

In [ ]:
DIR = cwd + '/Analysis_Results'
try:
    all_files = [name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))]
except FileNotFoundError:
    print ('Path', DIR, 'doesnt exist')
    sys.exit()

for i in all_files:
    if i[-4:] == '.pdf':
        pdfs.append(i)
pdfs.sort()

for i in range(0,len(pdfs)):
    pdfs[i] = DIR+'/'+pdfs[i]

merger = PdfFileMerger()

for pdf in pdfs:
    merger.append(pdf)

merger.write(cwd+'/Analysis_Results/00_All_Pages_Combined_'+'_'.join(lastnames)+'_Analysis.pdf')
merger.close()

for i in pdfs:
    os.remove(i)

script_exec_end = time.time()
print('Script Completed: Time taken is ', round((script_exec_end - script_exec_start),2), 'seconds')